In [1]:
from utils import Agent, Road, Trpf, read_config
from itertools import repeat
import numpy as np

# Functions

In [2]:
def get_route_choices(agents, route_count):
    choices = list(map(lambda agent: agent.choose_route(), agents))
    route_traveller_counts = np.bincount(choices, minlength = route_count)[:, np.newaxis]
    return choices, route_traveller_counts

def get_route_costs(roads, choices, route_traveller_counts):
    road_traveller_counts = np.array(route_to_road * route_traveller_counts)
    list(map(lambda count, road: road.add_travellers(count), road_traveller_counts[:,0], roads))
    road_costs = np.array(list(map(lambda road: road.report_cost(), roads)))[:, np.newaxis]
    route_costs = np.array(road_to_route * road_costs)
    return route_costs

def give_costs(agents, choices, route_costs):    
    list(map(lambda agent, choice: agent.recieve_travel_cost(route_costs[choice,0]), agents, choices))
    
def get_reports(agents, trpf, choices, excess_traveller_counts):
    reports = list(map(lambda agent, choice, excess: agent.report_congestion(excess[choice,0]),\
        agents, choices, repeat(excess_traveller_counts)))
    list(map(lambda route, report: trpf.recieve_report(route, report), choices, reports))
    
def give_trpfs(trpf, trpf_agents):
    route_trpfs = trpf.calculate_trpf()
    list(map(lambda agent, trpf: agent.recieve_trpf(trpf), trpf_agents, repeat(route_trpfs)))

# Read the configuration

In [3]:
simulation_folder = '/home/ihsan/Research/xfer/5/'
traffic_file = simulation_folder + 'traffic.properties'
roadnet_file = simulation_folder + 'roadnet.csv'

config = read_config(traffic_file, roadnet_file)

tresholds = config['tresholds']
weights = config['weights']
trpf_use_percent = config['trpf_use_percent']
t = config['t']
agent_count = 200 #config['agent_count']
change_percent = config['change_percent']
routes = config['routes']
route_opts = np.array(config['route_opts'])[:, np.newaxis]
round_count = 200 #config['round_count']
road_params = config['road_params']

# Initialize the simulation

In [4]:
route_count = len(routes)

trpf = Trpf(route_count, round_count, t)

roads = []
road_inds = {}
for i, params in enumerate(road_params.itertuples(index=False)):
    road = Road(*params)
    roads.append(road)
    road_name = params[0] + params[1] # Node 1 + Node 2
    road_inds[road_name] = i
    
agents = []
trpf_agents = []
for tag in range(agent_count):
    agent = Agent(tag, tresholds, weights, route_count, change_percent, trpf_use_percent)
    agents.append(agent)
    if agent.uses_trpf():
        trpf_agents.append(agent)
trpf_agents = trpf_agents

route_to_road = np.zeros((len(roads), len(routes)))
for i, route in enumerate(routes):
    for e in range(1, len(route)):
        road_name = route[e-1] + route[e]
        j = road_inds[road_name]
        route_to_road[j, i] = 1
route_to_road = np.matrix(route_to_road)
road_to_route = route_to_road.T 

historic_choices = np.zeros((round_count, route_count))
print('Number of trpf users: {}, Number of routes: {}, Number of roads:{}'.format(len(trpf_agents),\
    route_count, len(roads)))

Number of trpf users: 175, Number of routes: 3, Number of roads:7


# Simulation

In [5]:
while trpf.start_new_round():
    choices, route_traveller_counts = get_route_choices(agents, route_count)
    excess_traveller_counts = np.absolute(route_traveller_counts - route_opts)
    route_costs = get_route_costs(roads, choices, route_traveller_counts)
    give_costs(agents, choices, route_costs)
    get_reports(agents, trpf, choices, excess_traveller_counts)
    give_trpfs(trpf, trpf_agents)
    # Save the choices
    historic_choices[trpf.current_round,:] = route_traveller_counts[:,0]

# Results

In [6]:
means = np.mean(historic_choices, axis=0)
stds = np.std(historic_choices, axis=0)
print('Means: ', means)
print('Standard deviations:', stds, '\n')
print(historic_choices)

Means:  [  20.9     60.735  118.365]
Standard deviations: [  8.72525071  16.08492384  17.57161845] 

[[  71.   72.   57.]
 [  72.   73.   55.]
 [  59.   61.   80.]
 [  53.   50.   97.]
 [  46.   44.  110.]
 [  39.   41.  120.]
 [  32.   51.  117.]
 [  30.   57.  113.]
 [  24.   50.  126.]
 [  21.   46.  133.]
 [  27.   49.  124.]
 [  21.   79.  100.]
 [  17.   85.   98.]
 [  16.   67.  117.]
 [  15.   48.  137.]
 [  13.   35.  152.]
 [  15.   69.  116.]
 [  14.   82.  104.]
 [  12.   70.  118.]
 [  10.   60.  130.]
 [  12.   40.  148.]
 [  24.   43.  133.]
 [  21.   73.  106.]
 [  18.   98.   84.]
 [  16.   73.  111.]
 [  15.   60.  125.]
 [  14.   47.  139.]
 [  26.   56.  118.]
 [  24.   76.  100.]
 [  21.   62.  117.]
 [  19.   56.  125.]
 [  16.   46.  138.]
 [  27.   51.  122.]
 [  26.   76.   98.]
 [  24.   64.  112.]
 [  19.   54.  127.]
 [  17.   50.  133.]
 [  30.   50.  120.]
 [  40.   54.  106.]
 [  34.   46.  120.]
 [  31.   36.  133.]
 [  27.   44.  129.]
 [  20.   70.  11